In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data_utils
import torch.functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model architecture

In [143]:
class Encoder(nn.Module):
    def __init__(self, fp_size):
        super(Encoder, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(fp_size, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 256)
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.relu(self.fc2(out))
        out = self.relu(self.fc3(out))
        out = self.relu(self.fc4(out))
        return out
    
class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, drop_prob):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out,_ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out
        
        """x_init = torch.zeros(1,1,42).to(device) # ohe for start token
        while True:
            out, h = self.gru(x_init, h)
            out = self.fc(self.relu(out[:,-1]))
            out = self.softmax(out)
            if torch.argmax(out) == symbol_to_idx['[end]']:
                break
            elif torch.argmax(out) == torch.argmax(x_true):
                next_X = x_true.clone()"""

    

class GRUAE(nn.Module):
    def __init__(self, fp_size, input_size, hidden_size, output_size, num_layers, drop_prob=0.2):
        super(GRUAE, self).__init__()
        self.encoder = Encoder(fp_size=fp_size)
        self.decoder = Decoder(input_size=input_size, hidden_size=hidden_size, 
                               num_layers=num_layers, drop_prob=drop_prob, output_size=output_size)
    def forward(self, x):
        out = self.encoder(x) # (batch_size, 256)
        print(f'Encoded size = {out.size()}')
        out = self.decoder(out)
        return out
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.decoder.num_layers, batch_size, self.decoder.hidden_size).zero_().to(device)
        return hidden

# Data Prep

In [4]:
import selfies as sf
data = pd.read_csv('./GRU_data/selfies.csv', header=None, names=['selfies'])
alphabet = sf.get_alphabet_from_selfies(data.selfies)
alphabet.add("[nop]") # [nop] is a special padding symbol
alphabet.add("[start]")
alphabet.add("[end]")
alphabet = list(sorted(alphabet))
pad_to_len = max(sf.len_selfies(s) for s in data.selfies) + 5  # 5
symbol_to_idx = {s: i for i, s in enumerate(alphabet)}
idx2char = {i: s for i, s in enumerate(alphabet)}

In [5]:
alphabet

['[#Branch1]',
 '[#Branch2]',
 '[#C]',
 '[#N]',
 '[/C]',
 '[/N]',
 '[/O]',
 '[=Branch1]',
 '[=Branch2]',
 '[=C]',
 '[=N+1]',
 '[=N]',
 '[=O]',
 '[=Ring1]',
 '[=Ring2]',
 '[=S]',
 '[Br]',
 '[Branch1]',
 '[Branch2]',
 '[C@@H1]',
 '[C@@]',
 '[C@H1]',
 '[C@]',
 '[C]',
 '[Cl]',
 '[F]',
 '[I]',
 '[N+1]',
 '[NH1]',
 '[N]',
 '[O-1]',
 '[O]',
 '[P]',
 '[Ring1]',
 '[Ring2]',
 '[S]',
 '[\\C]',
 '[\\N]',
 '[\\O]',
 '[end]',
 '[nop]',
 '[start]']

In [6]:
import re
class SELFIESVectorizer:
    def __init__(self, alphabet, pad_to_len):
        self.alphabet = alphabet
        self.pad_to_len = pad_to_len
        self.char2idx = {s: i for i, s in enumerate(alphabet)}
        self.idx2char = {i: s for i, s in enumerate(alphabet)}
    def vectorize(self, selfie):
        ''' Vectorize a list of SMILES strings to a numpy array of shape (len(smiles), embed, len(charset))'''
        X = np.zeros((self.pad_to_len, len(self.alphabet)))
        splited = ['[start]'] + self.split_selfi(selfie) + ['[end]'] + ['[nop]'] * (self.pad_to_len - len(self.split_selfi(selfie)) - 2)
        for i, char in enumerate(splited):
            X[i, self.char2idx[char]] = 1
        return X
    def devectorize(self, ohe):
        ''' Devectorize a numpy array of shape (len(smiles), embed, len(charset)) to a list of SMILES strings'''
        selfie_str = ''
        for j in range(self.pad_to_len):
            char = self.idx2char[np.argmax(ohe[j])]
            if char == '[start]':
                continue
            elif char == '[end]':
                break
            else:
                selfie_str += char
        return selfie_str

    def split_selfi(self, selfie):
        pattern = r'(\[[^\[\]]*\])'
        return re.findall(pattern, selfie)

In [7]:
selfi_test = '[C][O][C][=C][C][=C][Branch2][Ring1][S][C][=C][N][=C][N][=C][Branch1][C][N][N][=C][Branch1][N][N][C][C][N][Branch1][C][C][C][C][Ring1][#Branch1][C][Ring1][=C][=N][Ring2][Ring1][C][C][=C][Ring2][Ring1][Branch2]'
vectorizer = SELFIESVectorizer(alphabet, pad_to_len)

In [8]:
vectorized = vectorizer.vectorize(selfi_test)
print(vectorized.shape)
print(vectorized)

(114, 42)
[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [9]:
devectorized = vectorizer.devectorize(vectorized)
print(devectorized)

[C][O][C][=C][C][=C][Branch2][Ring1][S][C][=C][N][=C][N][=C][Branch1][C][N][N][=C][Branch1][N][N][C][C][N][Branch1][C][C][C][C][Ring1][#Branch1][C][Ring1][=C][=N][Ring2][Ring1][C][C][=C][Ring2][Ring1][Branch2]


In [123]:
from torch.utils.data import Dataset, DataLoader

class GRUDataset(Dataset):
    def __init__(self, smiles_fp, selfies, vectorizer):
        self.smiles_fp = pd.read_csv(smiles_fp, sep=',', nrows=1000)
        self.selfies = pd.read_csv(selfies, nrows=1000)
        self.X = self.prepare_X(self.smiles_fp)
        self.X = np.array([self.reconstruct_fp(fp) for fp in self.X])
        #self.X = np.array([self.fp_to_matrix(fp) for fp in self.X])
        self.y = self.prepare_y(self.selfies)
    def __len__(self):
        return len(self.smiles_fp)
    def __getitem__(self, idx):
        raw_selfie = self.y[idx][0]
        vectorized_selfie = vectorizer.vectorize(raw_selfie)
        return torch.from_numpy(self.X[idx]).float(), torch.from_numpy(vectorized_selfie).float()


    @staticmethod
    def prepare_X(smiles_fp):
        fps = smiles_fp.fps.apply(eval).apply(lambda x: np.array(x, dtype=int))
        return fps
    @staticmethod
    def prepare_y(selfies):
        return selfies.values
    @staticmethod
    def reconstruct_fp(fp, length=4860):
        fp_rec = np.zeros(length)
        fp_rec[fp] = 1
        return fp_rec
    @staticmethod
    def fp_to_matrix(fp, length=4860):
        fp = np.array(fp)
        anti_fp = np.ones(4860) - fp
        return np.stack([fp, anti_fp], axis=1)

In [124]:
dataset = GRUDataset('GRU_data/chembl_klek.csv', 'GRU_data/selfies.csv', vectorizer)

In [125]:
dataset[0][0].shape, dataset[0][1].shape

(torch.Size([4860]), torch.Size([114, 42]))

In [126]:
dataset[0][0]

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [127]:
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [128]:
batch_size = 32
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [129]:
next(iter(train_loader))[0].shape

torch.Size([32, 4860])

# Train

In [146]:
def train(train_loader, learn_rate, EPOCHS=5):
    
    # Setting common hyperparameters
    input_size = 1
    hidden_size = 128
    num_layers = 1
    fp_size = 4860
    output_size = 42

    model = GRUAE(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,
                 output_size=output_size, fp_size=fp_size)
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of GRU model")
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.time()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            h = h.data
            model.zero_grad()
            print(f"x size = {x.size()}")
            out, h = model(x.to(device).float())
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.time()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

def evaluate(model, test_x, test_y, label_scalers):
    model.eval()
    outputs = []
    targets = []
    start_time = time.time()
    for i in test_x.keys():
        inp = torch.from_numpy(np.array(test_x[i]))
        labs = torch.from_numpy(np.array(test_y[i]))
        h = model.init_hidden(inp.shape[0])
        out, h = model(inp.to(device).float())
        outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
        targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
    print("Evaluation Time: {}".format(str(time.time()-start_time)))
    sMAPE = 0
    for i in range(len(outputs)):
        sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
    print("sMAPE: {}%".format(sMAPE*100))
    return outputs, targets, sMAPE

In [147]:
lr = 0.001
gru_model = train(train_loader, lr)

Starting Training of GRU model
x size = torch.Size([32, 4860])
Encoded size = torch.Size([32, 256])


RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 3-D tensor